In [ ]:
# Lines from   are taken from Reference [4]
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader


# train and validation data directory
train_data_dir = "/kaggle/input/facial-emotion-expressions/images/images/train/"
validation_data_dir = "/kaggle/input/facial-emotion-expressions/images/images/validation/"


# load the train and validation data
train_dataset = ImageFolder(train_data_dir,transform = transforms.Compose([
                transforms.ToTensor()
]))
validation_dataset = ImageFolder(validation_data_dir,transforms.Compose([
                transforms.ToTensor()
]))


# load the train and validation into batches.
train_dl = DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers = 4)
val_dl = DataLoader(validation_dataset, batch_size = 128, shuffle = False, num_workers = 4)




# Lines from   are taken from Reference [1]

# Architecture of our CNN Model
import torch.nn as nn # basic building block for neural networks
import torch.nn.functional as F # import convolution functions like Relu

class Net(nn.Module):
# Models a simple Convolutional Neural Network
	
    def __init__(self):
# initialize the network 
        super(Net, self).__init__() 
# 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
    
# Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
         
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 7)

    def forward(self, x):
# the forward propagation algorithm
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)


# Lines from   are taken from Reference [1]

#Define a Loss function and optimizer
import torch.optim as optim # optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)




# Train the network
train_losses=[]
train_accu=[]

for epoch in range(10):  # loop over the dataset multiple times
    print('Epoch : %d'%epoch)
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
       
        train_loss=running_loss/len(train_dl)
        accu=100.*correct/total
        #print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))
   
        train_accu.append(accu)
        train_losses.append(train_loss)
        
print('\nLoss for train dataset : %.3f | Accuracy for train dataset : %.3f'%(train_loss,accu))

            
# Testing on validation set  
for epoch in range(10):
    print('Epoch : %d'%epoch)
    eval_losses=[]
    eval_accu=[]
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_dl:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            test_loss=running_loss/len(val_dl)
            accu=100.*correct/total
            #print('Validation Loss: %.3f | Accuracy: %.3f'%(test_loss,accu))
            
            eval_losses.append(test_loss)
            eval_accu.append(accu)

print('\nLoss for validation dataset : %.3f | Accuracy for validation dataset : %.3f'%(test_loss,accu))


import matplotlib.pyplot as plt
# Plotting a line graph for Accuracy
plt.plot(train_accu,'-o')
plt.plot(eval_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')
 
plt.show()
    
# Plotting a line graph for Loss         
plt.plot(train_losses,'-o')
plt.plot(eval_losses,'-o')
plt.xlabel('epoch')
plt.ylabel('losses')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Losses')
 
plt.show()